<a href="https://colab.research.google.com/github/YASHGUPTA2611/Sppech-to-text/blob/main/Speech_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install --upgrade --no-deps --force-reinstall git+https://github.com/openai/whisper.git
!pip install yt-dlp
!pip install openai -q
!pip install ffmpeg-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-d7q3cm5k
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-d7q3cm5k
  Resolved https://github.com/openai/whisper.git to commit 248b6cb124225dd263bb9bd32d060b6517e067f8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=798075 sha256=98fa287f03be403163c0884154b43f57926866e716f8fac2fc0aad2fd6c1681b
  Stored in directory: /tmp/pip-ephem-wheel-cache-a1hn1knq/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
  Attempting uninstall: openai-whisper
    Found existing installation: openai-whisper 20230314
    Uninstalling openai-whisper-20230314

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
import whisper
import  openai
import yt_dlp
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

In [33]:
model = whisper.load_model('base')

## Function to use OpenAI whisper model which converts Speech to text

In [ ]:
def whisper_model(path):
  result = model.transcribe(path)
  return result['text']

## Converting audio to text using audio file inputed

In [ ]:
audio_file_path=''
audio_file_output = whisper_model(audio_file_path)

## Convert Speech to text from Youtube Video

In [48]:

youtube_link = 'https://www.youtube.com/watch?v=zP0f2duNZL0&t=1082s&ab_channel=AlphaStreetIndia'
def download_audio(url):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'outtmpl': '%(title)s.%(ext)s',
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
      info_dict = ydl.extract_info(url, download=False)
      filename = f"{info_dict['title']}.mp3"
      ydl.download([url])

    return filename
youtube_file_name = download_audio(youtube_link)
### Converting audio to text using youtube video
yotube_output = whisper_model(f'/content/{youtube_file_name}')

[youtube] Extracting URL: https://www.youtube.com/watch?v=zP0f2duNZL0&t=1082s&ab_channel=AlphaStreetIndia
[youtube] zP0f2duNZL0: Downloading webpage
[youtube] zP0f2duNZL0: Downloading android player API JSON
[youtube] Extracting URL: https://www.youtube.com/watch?v=zP0f2duNZL0&t=1082s&ab_channel=AlphaStreetIndia
[youtube] zP0f2duNZL0: Downloading webpage
[youtube] zP0f2duNZL0: Downloading android player API JSON
[info] zP0f2duNZL0: Downloading 1 format(s): 251
[dashsegments] Total fragments: 7
[download] Destination: HFCL Limited Q4 FY23 Earnings Concall.webm
[download] 100% of   61.26MiB in 00:00:03 at 16.33MiB/s              
[ExtractAudio] Destination: HFCL Limited Q4 FY23 Earnings Concall.mp3
Deleting original file HFCL Limited Q4 FY23 Earnings Concall.webm (pass -k to keep)


## Convert Speech to text from via input from MIC

In [42]:

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))
  fileName = 'recording.wav'  # Set the desired file name here
  with open(fileName, 'wb') as file:
    file.write(riff)

  print("File name:", fileName)

  return audio, sr

audio, sr = get_audio()
### Converting audio to text using input audio via MIC
Mic_output = whisper_model('/content/recording.wav')